# Churn prediction Pipeline

> Acá no voy a "profundizar" en el código, solo en el código nuevo, el resto está en el `churn_prediction.ipynb`

Librerias

In [57]:
from joblib import dump, load

import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from custom_transformers import YesNoTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline

from sklearn.base import clone


Cargo los datos y los limpio, la variable objetivo se puede procesar acá, no importa porque igual no la necesitamso en los pipelines

In [58]:
df = pd.read_csv('data/Churn.csv')

df = df.drop(columns=['customerID'])
df['TotalCharges'] = df['TotalCharges'].replace(' ', 0)
df['TotalCharges'] = df['TotalCharges'].astype(float)

df.columns = df.columns.str.lower().str.replace(' ', '_')

df['churn'] = df['churn'].map({"No": 0, "Yes": 1})

Train test split (para evaluar los datos)

In [59]:
X = df.drop('churn', axis=1).copy()
y = df['churn'].copy()

x_train, x_rest, y_train, y_rest = train_test_split(X, y, train_size=0.6, random_state=42, stratify=y)
x_valid, x_test, y_valid, y_test = train_test_split(x_rest, y_rest, test_size=0.5, random_state=42, stratify=y_rest)

print("Tamaño del conjunto de entrenamiento:", x_train.shape)
print("Tamaño del conjunto de validación:", x_valid.shape)
print("Tamaño del conjunto de prueba:", x_test.shape)

Tamaño del conjunto de entrenamiento: (4225, 19)
Tamaño del conjunto de validación: (1409, 19)
Tamaño del conjunto de prueba: (1409, 19)


Pipelines

In [60]:
yes_no = ColumnTransformer([
    ('yes_no',
    YesNoTransformer(),
    ['partner','dependents','phoneservice','paperlessbilling'])
])

one_hot_encoding = ColumnTransformer([
    ('one_hot_encode',
    OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    ['gender', 'multiplelines', 'internetservice', 'onlinesecurity',
        'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
        'streamingmovies', 'contract', 'paymentmethod'])
])

scaler = ColumnTransformer([
    ('scaler',
    MinMaxScaler(),
    ['tenure', 'monthlycharges', 'totalcharges'])
])

passthrough = ColumnTransformer([
    ('passthrough',
    'passthrough',
    ['seniorcitizen'])
])

In [61]:
feature_ingineering_pipe = Pipeline([
        (
            "features",
            FeatureUnion(
                [
                    ("yes_no", yes_no),
                    ("encoding", one_hot_encoding),                    
                    ("scaler", scaler),
                    ("pass", passthrough)
                ]
            ),
        )
])

In [62]:
model_pipe = Pipeline([
        ('Feature_engineering',clone(feature_ingineering_pipe)),
        ('model',LogisticRegression())
])

model testing

In [63]:
df_x = pd.concat([x_train,x_valid])
df_y = pd.concat([y_train, y_valid])

In [64]:
model_pipe.fit(df_x,df_y)

Pipeline(steps=[('Feature_engineering',
                 Pipeline(steps=[('features',
                                  FeatureUnion(transformer_list=[('yes_no',
                                                                  ColumnTransformer(transformers=[('yes_no',
                                                                                                   YesNoTransformer(),
                                                                                                   ['partner',
                                                                                                    'dependents',
                                                                                                    'phoneservice',
                                                                                                    'paperlessbilling'])])),
                                                                 ('encoding',
                                                                  ColumnTransformer(transformers=[('one_hot_encode',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse_output=Fals...
                                                                                                    'onlinebackup',
                                                                                                    'deviceprotection',
                                                                                                    'techsupport',
                                                                                                    'streamingtv',
                                                                                                    'streamingmovies',
                                                                                                    'contract',
                                                                                                    'paymentmethod'])])),
                                                                 ('scaler',
                                                                  ColumnTransformer(transformers=[('scaler',
                                                                                                   MinMaxScaler(),
                                                                                                   ['tenure',
                                                                                                    'monthlycharges',
                                                                                                    'totalcharges'])])),
                                                                 ('pass',
                                                                  ColumnTransformer(transformers=[('passthrough',
                                                                                                   'passthrough',
                                                                                                   ['seniorcitizen'])]))]))])),
                ('model', LogisticRegression())])

In [65]:
test_pred_y = model_pipe.predict(x_test)
print(f'accuracy: {accuracy_score(y_test,test_pred_y)}, recall: {recall_score(y_test,test_pred_y)}')

accuracy: 0.7849538679914834, recall: 0.5240641711229946


guardamos el modelo

In [66]:
dump(model_pipe, 'modelo_churn.joblib')

['modelo_churn.joblib']